# Analyse des schémas de comportement des utilisateurs dans les applications DeFI
## Analye des API Web 3 en vue de constituer un jeu de données

### Imports

In [46]:
pip install pandas web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import json
import pandas as pd

In [2]:
CG_API_KEY = 'CG-km5UvH2DKweLnkSnfmi73iRH'

### PROTOCOLS

In [40]:
import requests
import json
import time

# Liste des protocoles clés et leurs identifiants sur CoinGecko
KEY_PROTOCOLS = [
    "uniswap",          # DEX
    "curve-dao-token",  # DEX 
    "balancer",         # DEX
    "aave",             # Lending
    "maker",            # Lending
    "yearn-finance",    # Yield Farming
    "harvest-finance",  # Yield Farming
    "dai",              # Stablecoin
    "usd-coin",         # Stablecoin
    "tether",           # Stablecoin
    "nftfi",            # NFT-Fi (optionnel)
]

def get_defi_protocol_info(protocol_id):
    """
    Récupère les informations détaillées d'un protocole spécifique.
    """
    url = f"https://api.coingecko.com/api/v3/coins/{protocol_id}"
    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": CG_API_KEY  # Ajout de la clé API
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            platforms = data.get('platforms', {})
            protocol_info = {
                'name': data.get('name', 'N/A'),
                'type': infer_protocol_type(protocol_id),  # Déduction du type selon la liste fournie
                'blockchain_contracts': [
                    {'blockchain': blockchain, 'contract': contract}
                    for blockchain, contract in platforms.items()
                ],
                'website_url': data.get('links', {}).get('homepage', ['N/A'])[0],  # URL officielle
                'symbol': data.get('symbol', 'N/A'),  # Symbole du token associé
                'market_cap_rank': data.get('market_cap_rank', 'N/A'),  # Classement par capitalisation
                'description': data.get('description', {}).get('en', '')[:250],  # Description limitée à 250 caractères
            }
            return protocol_info
        elif response.status_code == 404:
            print(f"Protocole introuvable : {protocol_id}")
        elif response.status_code == 429:
            print("Trop de requêtes. Attente de 10 secondes...")
            time.sleep(10)  # Attendre 10 secondes en cas de limite d'API
            return get_defi_protocol_info(protocol_id)  # Réessayer
        else:
            print(f"Erreur inconnue avec {protocol_id}: {response.status_code}")
    except Exception as e:
        print(f"Erreur avec {protocol_id} : {str(e)}")
    return None

def infer_protocol_type(protocol_id):
    """
    Déduit le type du protocole en fonction de son identifiant.
    """
    if protocol_id in ["uniswap", "sushiswap", "curve-dao-token", "balancer"]:
        return "DEX"
    elif protocol_id in ["aave", "compound", "maker"]:
        return "Lending"
    elif protocol_id in ["yearn-finance", "harvest-finance", "curve-dao-token"]:
        return "Yield Farming"
    elif protocol_id in ["dai", "usd-coin", "tether"]:
        return "Stablecoin"
    elif protocol_id in ["nftfi", "nifty-gateway", "opensea"]:
        return "NFT-Fi"
    else:
        return "DeFi"

def save_protocols_to_json(protocols, filename):
    """
    Sauvegarde une liste de protocoles dans un fichier JSON.
    """
    with open(filename, 'w') as f:
        json.dump(protocols, f, indent=4)

# Processus principal
if __name__ == "__main__":
    print("Extraction des données pour les protocoles clés...")
    protocols_data = []
    for protocol_id in KEY_PROTOCOLS:
        protocol_info = get_defi_protocol_info(protocol_id)
        if protocol_info:
            protocols_data.append(protocol_info)
            # Respecter la limite de requêtes : attendre 2 secondes entre chaque requête
            time.sleep(2)

    print(f"{len(protocols_data)} protocoles récupérés.")
    
    # Sauvegarder dans un fichier JSON
    save_protocols_to_json(protocols_data, 'key_protocols.json')
    print("Données sauvegardées dans 'key_protocols.json'.")

Extraction des données pour les protocoles clés...
Protocole introuvable : sushiswap
Protocole introuvable : compound
Protocole introuvable : nifty-gateway
Protocole introuvable : opensea
11 protocoles récupérés.
Données sauvegardées dans 'key_protocols.json'.


### USERS

In [11]:
import json
from web3 import Web3
from datetime import datetime
import time

# Charger le fichier key_protocols.json
with open('key_protocols.json', 'r') as f:
    protocols_data = json.load(f)

# Configuration Web3 pour Ethereum (ex: Infura)
#infura_url = "https://mainnet.infura.io/v3/YOUR_INFURA_PROJECT_ID"  # Remplacez par votre propre URL Infura ou autre fournisseur
#web3 = Web3(Web3.HTTPProvider(infura_url))

#if not web3.isConnected():
#    raise ConnectionError("Impossible de se connecter à la blockchain Ethereum.")

# Fonction pour récupérer les transactions liées à un contrat sur une période donnée
def fetch_users_from_contract(contract_address, start_date, end_date):
    # Convertir les dates en timestamps Unix
    start_timestamp = int(time.mktime(start_date.timetuple()))
    end_timestamp = int(time.mktime(end_date.timetuple()))
    
    # Récupérer le bloc de départ
    start_block = web3.eth.get_block('earliest')
    end_block = web3.eth.get_block('latest')

    # Trouver les blocs qui couvrent cette période
    block_step = 5000  # On parcourt les blocs en petits groupes pour éviter de surcharger
    users = set()

    for block_number in range(start_block['number'], end_block['number'], block_step):
        # Itérer sur les transactions dans chaque bloc
        for block in range(block_number, block_number + block_step):
            block_data = web3.eth.get_block(block, full_transactions=True)
            for tx in block_data['transactions']:
                if tx['to'] == contract_address:  # Vérifier si la transaction concerne le contrat
                    timestamp = block_data['timestamp']
                    if start_timestamp <= timestamp <= end_timestamp:
                        users.add(tx['from'])  # Ajouter l'utilisateur à l'ensemble
    
    return [{"address": user, "blockchain": "ethereum"} for user in users]

# Date de début et de fin pour décembre 2024
start_date = datetime(2024, 12, 1)
end_date = datetime(2024, 12, 31, 23, 59, 59)

# Extraire les utilisateurs associés aux contrats
users_data = []
for protocol in protocols_data:
    blockchain_contracts = protocol.get("blockchain_contracts", [])
    
    for contract_info in blockchain_contracts:
        contract_address = contract_info.get("contract")
        blockchain = contract_info.get("blockchain")

        print(contract_address)
        print(blockchain)

    
    if contract_address and blockchain:
        # Convertir l'adresse du contrat en format checksum
        contract_address = web3.toChecksumAddress(contract_address)
        
        # Récupérer les utilisateurs associés à ce contrat
        users = fetch_users_from_contract(contract_address, start_date, end_date)
        
        for user in users:
            user_data = {
                "user_id": None,  # Générer un ID unique dans une base réelle
                "address": user["address"],
                "blockchain": user["blockchain"],
                "created_at": datetime.utcnow().isoformat()  # Date actuelle en format ISO
            }
            users_data.append(user_data)

# Sauvegarder les utilisateurs extraits dans un fichier JSON
with open('users_december_2024.json', 'w') as f:
    json.dump(users_data, f, indent=4)

# Afficher les 5 premiers utilisateurs pour vérifier
print(json.dumps(users_data[:5], indent=4))

0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
ethereum
0x4537e328bf7e4efa29d05caea260d7fe26af9d74
xdai
1f9840a85d5af5bf1d1762f925bdaddc4201f984.factory.bridge.near
near-protocol
0x6fd9d7ad17242c41f7131d257212c54a0e816691
optimistic-ethereum
0x22c54ce8321a4015740ee1109d9cbc25815c46e6
huobi-token
0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0
arbitrum-one
0xb33eaad8d922b1083446dc23f610c2567fb5180f
polygon-pos
0x90d81749da8867962c760414c1c25ec926e889b6
harmony-shard-0
0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
avalanche
0x665b3a802979ec24e076c80025bff33c18eb6007
energi
0x009be848df92a400da2f217256c88d1a9b1a0304f9b3e90991a67418e1d3b08c
sora
0xbf5140a22578168fd562dccf235e5d43a02ce9b1
binance-smart-chain


NameError: name 'web3' is not defined